In [2]:
# ==== Transfer Learning with Pre-Extracted Dataset ====

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# ==== Dataset directories ====
train_dir = r"C:\Users\Sayali\Downloads\cats_and_dogs_filtered\cats_and_dogs_filtered\train"
val_dir = r"C:\Users\Sayali\Downloads\cats_and_dogs_filtered\cats_and_dogs_filtered\validation"

# ==== Load datasets ====
img_height, img_width = 224, 224
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [3]:
# ✅ Get class names BEFORE prefetch/cache
class_names = train_ds.class_names
print("Classes:", class_names)

# ✅ Then apply performance optimizations
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)


Classes: ['cats', 'dogs']


In [ ]:
# ==== Model ====
base_model = tf.keras.applications.EfficientNetV2B0(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

epochs = 10
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 18s 1us/step
Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 82s 994ms/step - accuracy: 0.9396 - loss: 0.1326 - val_accuracy: 0.9960 - val_loss: 0.0123
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.9942 - loss: 0.0157 - val_accuracy: 0.9960 - val_loss: 0.0120
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 62s 988ms/step - accuracy: 0.9969 - loss: 0.0057 - val_accuracy: 0.9970 - val_loss: 0.0103
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 53s 843ms/step - accuracy: 0.9992 - loss: 0.0047 - val_accuracy: 0.9930 - val_loss: 0.0187
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 54s 858ms/step - accuracy: 0.9950 - loss: 0.0168 - val_accuracy: 0.9960 - val_loss: 0.0113
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 58s 926ms/step - accuracy: 0.9979 - loss: 0.0067 - val_accuracy: 0.9960 - val_loss: 0.0146
Epoch 7/10
46/63 ━━━━━━━━━━━━━━━━━━━━ 11s 669ms/step - accuracy: 0.9971 - loss: 0.0091

In [ ]:
# ==== Plot Accuracy ====
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel("Epochs"); plt.ylabel("Accuracy")
plt.legend(); plt.show()


In [ ]:
# ==== Evaluate ====
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {acc*100:.2f}%")


In [ ]:
# ==== Show Predictions ====
for images, labels in val_ds.take(1):
    preds = model.predict(images)
    pred_labels = tf.argmax(preds, axis=1)
    
    plt.figure(figsize=(10,10))
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Pred: {class_names[pred_labels[i]]}\nTrue: {class_names[labels[i]]}")
        plt.axis("off")
    plt.show()